In [8]:
import cv2

# Load the pre-trained Haar cascade for eye detection
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Function to detect eyes in an image and return their positions
def detect_eyes(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))
    print(f"This is eye{eyes}")
    return eyes

# Function to estimate the gaze direction based on the eye positions
def estimate_gaze(eyes, image):
    height, width, _ = image.shape
    eye_centers = []
    for (x, y, w, h) in eyes:
        center_x = x + w // 2
        center_y = y + h // 2
        eye_centers.append((center_x, center_y))

        # Draw bounding boxes around the eyes
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.circle(image, (center_x, center_y), 2, (0, 0, 255), 2)
        
    # Estimate the gaze direction based on the eye positions
    # if len(eye_centers) == 2:
    #     eye1_x, eye1_y = eye_centers[0]
    #     eye2_x, eye2_y = eye_centers[1]

    #     # Calculate the average of eye coordinates as the point of gaze
    #     gaze_x = (eye1_x + eye2_x) // 2
    #     gaze_y = (eye1_y + eye2_y) // 2

    #     print(f"User is looking at coordinates: ({gaze_x}, {gaze_y})")

    #     # Determine the gaze direction based on the differences in x and y coordinates
    #     if gaze_y < height // 3:
    #         # Look up
    #         print("User is looking up")
    #         cv2.circle(image, (gaze_x, gaze_y), 10, (0, 0, 255), -1)
    #     elif gaze_y > (2 * height) // 3:
    #         # Look down
    #         print("User is looking down")
    #         cv2.circle(image, (gaze_x, gaze_y), 10, (0, 0, 255), -1)
    #     elif gaze_x < width // 3:
    #         # Look left
    #         print("User is looking left")
    #         cv2.circle(image, (gaze_x, gaze_y), 10, (0, 0, 255), -1)
    #     elif gaze_x > (2 * width) // 3:
    #         # Look right
    #         print("User is looking right")
    #         cv2.circle(image, (gaze_x, gaze_y), 10, (0, 0, 255), -1)
    #     else:
    #         # Look straight ahead
    #         print("User is looking straight ahead")
    #         cv2.circle(image, (gaze_x, gaze_y), 10, (0, 0, 255), -1)

    return image

# Main loop to capture video from the webcam and detect eyes
def main():
    # Open the webcam
    cap = cv2.VideoCapture(0)

    while True:
        # Read a frame from the webcam
        ret, frame = cap.read()

        # Flip the frame horizontally for a mirror effect
        frame = cv2.flip(frame, 1)

        # Detect eyes in the frame
        eyes = detect_eyes(frame)

        # Estimate gaze direction and draw visualizations
        output_frame = estimate_gaze(eyes, frame)

        # Display the resulting frame
        cv2.imshow('Eye Detection', output_frame)

        # Exit the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close the OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


This is eye[[349 266  62  62]]
This is eye[[347 257  76  76]]
This is eye[[347 255  73  73]]
This is eye[[349 251  69  69]]
This is eye[[357 229  71  71]
 [284 232  73  73]]
This is eye[[353 225  83  83]]
This is eye[[357 230  73  73]]
This is eye[[356 227  74  74]]
This is eye[[355 224  75  75]]
This is eye[[357 225  71  71]
 [285 233  71  71]]
This is eye[[356 227  72  72]]
This is eye[[359 226  72  72]
 [285 229  76  76]]
This is eye[[361 224  70  70]
 [285 227  77  77]]
This is eye[[287 223  75  75]
 [360 218  75  75]]
This is eye[[359 218  73  73]
 [284 222  73  73]]
This is eye[[359 217  69  69]
 [292 218  72  72]]
This is eye()
This is eye()
This is eye()
This is eye()
This is eye()
This is eye()
This is eye()
This is eye()
This is eye()
This is eye()
This is eye()
This is eye()
This is eye()
This is eye()
This is eye()
This is eye()
This is eye()
This is eye[[346 230  73  73]]
This is eye()
This is eye()
This is eye()
This is eye[[346 240  70  70]]
This is eye[[345 247  70  70]

In [3]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
model = load_model('eye_model.h5')
folders = ['bottom_left', 'bottom_right', 'bottom_middle', 'top_left', 'top_right', 'top_middle', 'left_middle', 'right_middle']



# Load your pre-trained CNN model

# Define the desired dimensions
desired_width = 250
desired_height = 50

# Initialize the webcam
video_capture = cv2.VideoCapture(0)  # 0 represents the default webcam device

while True:
    # Capture frame-by-frame from the webcam
    ret, frame = video_capture.read()

    # Flip the frame horizontally
    # frame = cv2.flip(frame, 1)

    # Define the ROI coordinates for the eye portion
    eye_x = 100
    eye_y = 100
    eye_width = 150
    eye_height = 50

    # Extract the eye ROI from the frame
    eye_roi = frame[eye_y:eye_y+eye_height, eye_x:eye_x+eye_width]

    # Resize the ROI to match the desired dimensions
    resized_roi = cv2.resize(eye_roi, (desired_width, desired_height))

    # Convert the ROI to RGB format
    resized_roi = cv2.cvtColor(resized_roi, cv2.COLOR_BGR2RGB)

    # Convert the ROI to numpy array
    resized_roi = np.asarray(resized_roi)

    # Expand dimensions to match the input shape of your CNN model
    resized_roi = np.expand_dims(resized_roi, axis=0)

    # Normalize pixel values
    resized_roi = resized_roi / 255.0

    # Pass the preprocessed ROI through your CNN model for prediction
    predictions = model.predict(resized_roi)
    predicted_class = np.argmax(predictions, axis=1)

    # Display the predicted class
    print(folders[predicted_class[0]])
    # Display the frame with overlays or visual indicators of person detection
    cv2.imshow('Eye Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the windows
video_capture.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 89ms/step
bottom_middle
1/1 [==============================] - 0s 63ms/step
bottom_middle
1/1 [==============================] - 0s 50ms/step
bottom_middle
1/1 [==============================] - 0s 34ms/step
bottom_middle
1/1 [==============================] - 0s 35ms/step
bottom_middle
1/1 [==============================] - 0s 31ms/step
bottom_middle
1/1 [==============================] - 0s 34ms/step
bottom_middle
1/1 [==============================] - 0s 29ms/step
bottom_middle
1/1 [==============================] - 0s 33ms/step
bottom_middle
1/1 [==============================] - 0s 37ms/step
bottom_middle
1/1 [==============================] - 0s 32ms/step
bottom_middle
1/1 [==============================] - 0s 31ms/step
bottom_middle
1/1 [==============================] - 0s 36ms/step
bottom_middle
1/1 [==============================] - 0s 35ms/step
bottom_middle
1/1 [==============================] - 0s 30ms/step
bottom_middle
1/1 [=====

In [7]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

model = load_model('eye_model.h5')
folders = ['bottom_left', 'bottom_right', 'bottom_middle', 'top_left', 'top_right', 'top_middle', 'left_middle', 'right_middle']

eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Load your pre-trained CNN model
def detect_eyes(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))
    return eyes


# Define the desired dimensions
desired_width = 250
desired_height = 50

# Initialize the webcam
video_capture = cv2.VideoCapture(0)  # 0 represents the default webcam device

while True:
    # Capture frame-by-frame from the webcam
    ret, frame = video_capture.read()
    eyes = detect_eyes(frame)

    if len(eyes) >= 2:
        
        (eye_x, eye_y, eye_width, eye_height), _ = eyes[:2]
        eye_roi = frame[eye_y:eye_y+eye_height, eye_x:eye_x+eye_width]

        # Resize the ROI to match the desired dimensions
        resized_roi = cv2.resize(eye_roi, (desired_width, desired_height))

        # Convert the ROI to RGB format
        resized_roi = cv2.cvtColor(resized_roi, cv2.COLOR_BGR2RGB)

        # Convert the ROI to numpy array
        resized_roi = np.asarray(resized_roi)

        # Expand dimensions to match the input shape of your CNN model
        resized_roi = np.expand_dims(resized_roi, axis=0)

        # Normalize pixel values
        resized_roi = resized_roi / 255.0

        # Pass the preprocessed ROI through your CNN model for prediction
        predictions = model.predict(resized_roi)
        predicted_class = np.argmax(predictions, axis=1)
        for (eye_x, eye_y, eye_width, eye_height) in eyes:
            frame = cv2.rectangle(frame, (eye_x, eye_y), (eye_x+eye_width, eye_y+eye_height), (0, 255, 0), 2)
        frame = cv2.putText(frame, folders[predicted_class[0]], (eye_x, eye_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame with overlays or visual indicators of eye detection
    cv2.flip(frame, 1)
    
    cv2.imshow('Eye Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the windows
video_capture.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 27ms/step
